In [2]:
import torch
from torch import nn
import torch.nn.functional as F

# 卷积
----------------
## 普通卷积

### 实现:

#### 官方实现
`torch.nn.Conv2d(in_chanels,out_channels,kernel_size,bias,stride,padding,padding_mode)`

**in_channels :**输入通道数目

**out_channels ：**输出通道数目

**kernel_size ：**卷积核大小，如果输入是一个值，比如 3 33，那么卷积核大小就是 3 × 3 3 \times 33×3 ，如果不想卷积核宽和高相等，还可以输入tuple类型数据，比如： ( 3 , 5 ) (3, 5)(3,5)

**stride ：**步长大小，跟上面卷积核参数一样，如果输入是一个值，比如 2 22 ，步长就是 2 × 2 2 \times 22×2 ，还可以输入元组 ( 2 , 1 ) (2, 1)(2,1) ，表示卷积核每次向右移动 1 11 个步长，向下移动 2 22 个步长。

**padding ：**填充，参数表示在周围补0的情况。补0的方向为上、下、左、右四个方向。如果是输入是单个值，比如1，就是在上下左右四个方向补一圈0。如果输入是元组比如 (2,1) ，表示在上下两个方向各补两行0，在左右两个方向各补一列0。

**bias ：**偏置，布尔类型，默认为 True ，即增加一个学习的偏置项。

**padding_mode ：**填充的模式，默认是 zero ，还可以选择 reflect 、 replicate 、 circular 。（有需要自行深入了解）
* valid padding（有效填充）：完全不使用填充。
* half/same padding（半填充/相同填充）：保证输入和输出的feature map尺寸相同。
* full padding（全填充）：在卷积操作过程中，每个像素在每个方向上被访问的次数相同。
* arbitrary padding（任意填充）：人为设定填充。

In [3]:
#手动实现
def conv2d(x, weight, bias, stride = 1 , pad = 0):      
    #参数x:输入; weight:权重; bias:偏重; stride:步长; pad:填充;
    n, c, h_in, w_in = x.shape
    d, c, k, j = weight.shape
    x_pad = torch.zeros(n, c, h_in+2*pad, w_in+2*pad)   # 对输入进行补零操作
    if pad>0:
        x_pad[:, :, pad:-pad, pad:-pad] = x
    else:
        x_pad = x

    x_pad = x_pad.unfold(2, k, stride)
    x_pad = x_pad.unfold(3, j, stride)           # 按照滑动窗展开
    out = torch.einsum(                          # 按照滑动窗相乘，
        'nchwkj,dckj->ndhw',                     # 并将所有输入通道卷积结果累加
        x_pad, weight)
    out = out + bias.view(1, -1, 1, 1)           # 添加偏置值
    return out

## 跨步卷积
跨步卷积主要改进的地方为，对滑动窗口的步幅进行调整，从原先的固定为1，转换为移动stride长度。

**跨步卷积去除了部分冗余且缩小了输出的尺寸，一定程度上取到了池化层的作用**

实现方式同上，但是将Stride设置不为1即可。

## 空洞卷积
空洞卷积中有一个额外的特殊参数扩张率对卷积核中相邻的数值进行填充

#### 官方实现
`torch.nn.Conv2d(in_chanels,out_channels,kernel_size,bias,dilation)`

**dilation：** 空洞率 

## 分组卷积
分组卷积对卷积层的通道进行分离成n组然后拼接。

#### 官方实现
`torch.nn.Conv2d(in_channels,out_channels,kernel_size,bias,groups)`

**groups：** 分组数

## 深度(可分离)卷积
包括深度卷积和逐点卷积

In [4]:
 def fixed_padding(inputs, kernel_size, dilation):
    # 调整输入的填充
    kernel_size_effective = kernel_size + (kernel_size - 1) * (dilation - 1)
    pad_total = kernel_size_effective - 1
    pad_beg = pad_total // 2
    pad_end = pad_total - pad_beg
    padded_inputs = F.pad(inputs, (pad_beg, pad_end, pad_beg, pad_end))
    return padded_inputs

class SeparableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, dilation=1, bias=False, BatchNorm=None):
        super(SeparableConv2d, self).__init__()
 
        #逐通道卷积，groups代表通道分组数，对通道进行分组，普通卷积默认是1
        #每个分组的输入通道是in_channels/groups，输出通道是groups/out_channels，需要整除。最后将每个分组的out_channels进行concat，得到最终的输出通道数
        #第一层卷积为深度卷积
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size, stride, 0, dilation,
                               groups=inplanes, bias=bias)
        
        self.bn = BatchNorm(inplanes)
        #逐点卷积
        self.pointwise = nn.Conv2d(in_channels, out_channels, 1, 1, 0, 1, 1, bias=bias)
 
    def forward(self, x):
        x = fixed_padding(x, self.conv1.kernel_size[0], dilation=self.conv1.dilation[0])
        x = self.conv1(x)
        x = self.bn(x)
        x = self.pointwise(x)
        return x

## 转置卷积

### 官方实现
`torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1, padding_mode='zeros', device=None, dtype=None)`

**padding** 在传统卷积中，我们的 padding 范围为 [ 0 , k − 1 ]，p = 0 被称为 No padding，p = k − 1 被称为 Full Padding。而在反卷积中的 p'刚好相反，也就是 p ′ = k − 1 − p。也就是当我们传 p ′ = 0 时，相当于在传统卷积中传了 p = k − 1，而传 p ′ = k − 1 时，相当于在传统卷积中传了 p = 0。

在反卷积中 p ′不能无限大，最大值为 k − 1 − p。（其实也不是哦）当大于限度值时就会为padding负数，即对输入图像进行裁剪。

**kernel_size** 反卷积进行卷积时，使用的参数是kernel的转置。

**sride** 在反卷积中，stride参数就是表示往输入图片每两个像素点中间填充0，而填充的数量就是 stride - 1。

**output_padding** output_padding的作用就是：在输出图像右侧和下侧补值，用于弥补stride大于1带来的缺失。其中output_padding必须小于stride.

**注** 如果卷积和反卷积的参数一致，卷积会让 A 尺寸变为 B 尺寸，那么反卷积就会将 B 尺寸变为 A 尺寸。